In [3]:
import emzed

# minimal workflow for peak picking and peak shape fitting

In [4]:
pm = emzed.io.loadPeakMap("example1.mzXML")
print pm.mzRange(), pm.rtRange()

(202.00143432617188, 649.9967041015625) (725.765930175781, 1212.01330566406)


In [5]:
pm = pm.extract(mzmin=300, mzmax=500)
print pm.mzRange(), pm.rtRange()

(300.00653076171875, 499.9892578125) (725.765930175781, 1212.01330566406)


In [6]:
peaks = emzed.ff.runMetaboFeatureFinder(pm, epdet_max_fwhm=30.0, epdet_min_fwhm=5.0)


========================== RUN FEATURE FINDER METABO ==========================

COMMON PARAMETERS

common_chrom_fwhm ................. 5.0
common_chrom_peak_snr ............. 3.0
common_noise_threshold_int ........ 10.0

PARAMS MASS TRACE DETECTION:

mtd_mass_error_ppm ................ 20.0
mtd_max_trace_length .............. 300.0
mtd_min_sample_rate ............... 0.5
mtd_min_trace_length .............. 5.0
mtd_reestimate_mt_sd .............. true
mtd_trace_termination_criterion ... outlier
mtd_trace_termination_outliers .... 5

PARAMS ELUTION PEAK DETECTION:

epdet_masstrace_snr_filtering ..... false
epdet_max_fwhm .................... 30.0
epdet_min_fwhm .................... 5.0
epdet_width_filtering ............. auto

PARAMS FEATURE FINDER METABO:

ffm_charge_lower_bound ............ 1
ffm_charge_upper_bound ............ 3
ffm_disable_isotope_filtering ..... false
ffm_isotope_model ................. metabolites
ffm_isotope_noisemodel ............ 5%RMS
ffm_local_mz_range .....

/Users/uweschmitt/Projects/emzed2/src/emzed/ff/_metaboff.py:246: UserWarning: you did not provide a type_ parameter, this might be dangerous !!!!
  tab.replaceColumn("mz", recalc)



======================== NEEDED 0 MINUTES AND 4 SECONDS =======================



In [7]:
peaks = emzed.utils.integrate(peaks, n_cpus=4)


integrate table using 4 processes

0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100
needed 1.9 seconds


In [8]:
%%capture
peaks.setColFormat("peakmap", "%s")
emzed.gui.inspect(peaks)

True

## R Interface

In [9]:
rip = emzed.r.RInterpreter()

rip.execute("x <- 3")
print rip.x

3


In [10]:
t = emzed.io.loadCSV("peaks.csv")
t = t[:20]
print(t)

id       feature_id mz        rt       intensity   fwhm     z        source        
int      int        float     float    float       float    int      str           
------   ------     ------    ------   ------      ------   ------   ------        
0        0          505.98672 15.37m   23400000.00 16.20    1        example1.mzXML
1        0          506.99009 15.37m   23400000.00 16.20    1        example1.mzXML
2        0          507.99120 15.37m   23400000.00 16.20    1        example1.mzXML
3        1          332.05748 13.18m   13600000.00 16.80    0        example1.mzXML
4        2          615.72842 15.71m   4560000.00  13.80    2        example1.mzXML
5        2          616.23003 15.71m   4560000.00  13.80    2        example1.mzXML
6        2          616.73102 15.71m   4560000.00  13.80    2        example1.mzXML
7        3          372.08195 13.14m   12900000.00 16.20    0        example1.mzXML
8        4          580.09612 13.65m   11400000.00 15.60    0        example

In [11]:
rip.t = t
rip.execute("print(t)")

   id feature_id       mz    rt intensity fwhm z         source
0   0          0 505.9867 922.2  23400000 16.2 1 example1.mzXML
1   1          0 506.9901 922.2  23400000 16.2 1 example1.mzXML
2   2          0 507.9912 922.2  23400000 16.2 1 example1.mzXML
3   3          1 332.0575 790.8  13600000 16.8 0 example1.mzXML
4   4          2 615.7284 942.6   4560000 13.8 2 example1.mzXML
5   5          2 616.2300 942.6   4560000 13.8 2 example1.mzXML
6   6          2 616.7310 942.6   4560000 13.8 2 example1.mzXML
7   7          3 372.0820 788.4  12900000 16.2 0 example1.mzXML
8   8          4 580.0961 819.0  11400000 15.6 0 example1.mzXML
9   9          5 265.0417 768.6  10700000 16.8 0 example1.mzXML
10 10          6 259.0216 769.8   9560000 17.4 1 example1.mzXML
11 11          6 260.0249 769.8   9560000 17.4 1 example1.mzXML
12 12          7 565.0458 819.0   9020000 15.6 1 example1.mzXML
13 13          7 566.0491 819.0   9020000 15.6 1 example1.mzXML
14 14          7 567.0504 819.0   902000

In [12]:
rip.execute("""

        ag1 <- aggregate(x=t$mz, by=list(t$feature_id), min);
        ag2 <- aggregate(x=t$mz, by=list(t$feature_id), max);
        result <- data.frame(feature_id=ag1$Group, mzmin=ag1$x, mzmax=ag2$x);

        """)

+ 
+         ag1 <- aggregate(x=t$mz, by=list(t$feature_id), min);
+         ag2 <- aggregate(x=t$mz, by=list(t$feature_id), max);
+         result <- data.frame(feature_id=ag1$Group, mzmin=ag1$x, mzmax=ag2$x);
+ 
+         })


In [13]:
print rip.result

feature_id mzmin      mzmax     
int        float      float     
------     ------     ------    
0          505.986720 507.991200
1          332.057480 332.057480
2          615.728420 616.731020
3          372.081950 372.081950
4          580.096120 580.096120
5          265.041690 265.041690
6          259.021610 260.024890
7          565.045810 567.050360
8          623.129250 623.129250
9          482.959610 483.962980
10         595.661380 596.163020

